In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
from simulation import get_setup

distance = 5

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
for yaw_deg, marker in zip([0,15,30], ["o", "*", "x"]):
    source, mic_positions = get_setup(distance_cm=distance, yaw_deg=yaw_deg)
    source_image = [source[0], -source[1]]
    for i, mic in enumerate(mic_positions):
        label = f"mic{i}" if yaw_deg == 0 else None
        ax.scatter(*mic*100, label=label, marker=marker, color=f"C{i}")
    ax.scatter(*mic*100, label=f"yaw {yaw_deg}deg", marker=marker, color=f"C{i}")
ax.plot([4.8*100, 5.2*100], [0, 0], color="k", label="wall")
ax.set_title(f"setup for distance={distance}cm and different angles")

handles, labels = ax.get_legend_handles_labels()
h_mics = {l: h for h, l in zip(handles, labels) if 'mic' in l}
l1 = ax.legend(h_mics.values(), h_mics.keys(),loc="lower right", bbox_to_anchor=[.3, 0])

h_other = {l: h for h, l in zip(handles, labels) if not 'mic' in l}
ax.legend(h_other.values(), h_other.keys(),loc="lower left", bbox_to_anchor=[.3, 0])
ax.add_artist(l1)

ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.axis("equal")

fig.savefig(f'plots/setup.pdf', bbox_inches='tight')

# Experimental distance-frequency matrix

In [ ]:
# exp_name = '2021_02_09_wall_tukey';
# exp_name = '2021_02_09_wall';

#exp_name = "2021_02_23_wall";  # old buzzer
exp_name = "2021_02_25_wall"; # new buzzer
#exp_name = "2021_03_01_flying"
# exp_name = '2020_12_9_rotating';
# exp_name = '2020_11_26_wall';
# fname = f'results/{exp_name}_real.pkl'

overwrite = False # generate new results
mic_type = "audio_deck"
motors = "all45000"

# 1. amplitude study

In [ ]:
from pandas_utils import filter_by_dict
from wall_detector import WallDetector
from generate_df_results import wall_detector_from_df

wall_detector = WallDetector()
backup_exists = wall_detector.fill_from_backup(exp_name, mic_type, motors)

if (not backup_exists) or overwrite:
    fname = f"../experiments/{exp_name}/all_data.pkl"
    # fname = f'../experiments/{exp_name}/battery_data.pkl' # for 2021_02_09_wall, battery study
    try:
        df_all = pd.read_pickle(fname)
        print("read", fname)
    except Exception as e:
        print(e)
        print("Error: run wall_analysis.py to parse experiments.")
    wall_detector = wall_detector_from_df(df_all, exp_name, mic_type, motors)
    wall_detector.backup(exp_name, mic_type, motors)

In [ ]:
wall_detector.df

In [ ]:
from calibration import fit_distance_slice
from simulation import get_amplitude_function

all_distances = wall_detector.df.distance.unique()

for mic, df_mic in wall_detector.df.groupby('mic'):
    mic_idx = int(mic)
    for i, (frequency, df_here) in enumerate(df_mic.groupby('frequency')):
        label = f"{frequency:.0f}Hz"
        
        # does not make a difference
        # df_here = df_here[df_here.magnitude > 2]
        distances = df_here.distance.unique()
        if len(distances) < len(all_distances): # only plot distances with "full coverage"
            continue
        
        fig, ax_fit = plt.subplots()
        fig.set_size_inches(7, 5)
        # fit the attenuation for this frequency 
        for d, series in df_here.groupby('distance').magnitude: 
            ax_fit.scatter([d]*len(series), series.values, color='C0', s=5.0)
        ax_fit.scatter([], [], color='C0', s=2.0, label='raw')
        mag_series = df_here.groupby('distance').magnitude.median()
        ax_fit.plot(mag_series.index, mag_series.values, color='C1', label='median')
        
        mags = pd.pivot_table(df_here, values='magnitude', index='distance', columns='counter', 
                              fill_value=0.0)
        distances_raw = np.array(mags.index.values, dtype=float)
        coeffs_raw, d_slice_raw, cost_raw = fit_distance_slice(
            mags.values[:, None, :], 
            distances_raw, method='minimize',
            yaw_deg=0, frequency=frequency, chosen_mics=[mic_idx], 
            optimize_absorption=True)
        ax_fit.plot(distances_raw, d_slice_raw, color='C2', label='fit to raw')
        alpha, phase, gain = coeffs_raw
        ax_fit.set_title(label)
        print(f'gain at {frequency:.0f}Hz: {gain:.2f}, median: {df_here.magnitude.median():.2f}')
        
        distances_median = mag_series.index
        coeffs_median, d_slice_median, cost_median = fit_distance_slice(
            mag_series.values[:, None], 
            distances_median,method='minimize', optimize_absorption=True,
            yaw_deg=0, frequency=frequency, chosen_mics=[mic_idx])
        ax_fit.plot(distances_median, d_slice_median, color='C3', ls=':', label='fit to median')
        #fit_distance_slice()
        ax_fit.legend(loc='upper right')
        
        # find the sigma for this frequency (per distance, and overall)
        std_series = df_here.groupby('distance').magnitude.std()
        amps = get_amplitude_function(std_series.index, 
                                      gain, 
                                      alpha, mic_idx)
        fig, ax_freq = plt.subplots()
        ax_freq.semilogy(std_series.index, std_series.values, label=label, color=f'C{i}')
        ax_freq.semilogy(std_series.index, amps*0.5, ls=':', color=f'C{i}')
        ax_freq.semilogy(std_series.index, amps, ls=':', color=f'C{i}')
        ax_freq.semilogy(std_series.index, amps*1.5, ls=':', color=f'C{i}')
        ax_freq.legend()
        ax_freq.set_xlabel('distance [cm]')
        ax_freq.set_ylabel('magnitude vs. std [-]')
    break

# DF analysis

In [ ]:
from wall_detector import prune_df_matrix
df_matrix_raw, df_dist, df_freq_raw = wall_detector.get_df_matrix()
df_matrix, df_freq, indices = prune_df_matrix(
    df_matrix_raw, df_freq_raw, ratio_missing_allowed=0.5, verbose=True
)

In [ ]:
from plotting_tools import add_colorbar

fig, ax_all = plt.subplots()
for mic_idx in range(4):
    fig, ax = plt.subplots()
    im = ax.pcolormesh(df_dist, df_freq_raw, df_matrix_raw[mic_idx])
    ax.set_title(f"mic{mic_idx} original")
    add_colorbar(fig, ax, im)
    fig, ax = plt.subplots()
    im = ax.pcolormesh(df_dist, df_freq, df_matrix[mic_idx])
    ax.set_title(f"mic{mic_idx} pruned")
    ax_all.plot(df_freq, np.nanmean(df_matrix[mic_idx], axis=1), label=f'mic{mic_idx}')
    add_colorbar(fig, ax, im)

# 1. study distance slices

In [ ]:
from simulation import get_dist_slice_theory
from copy import deepcopy
from pandas_utils import fill_nans

def plot_ffts(slice_exp, slice_the, dist):
    fig_f, axs_f = plt.subplots(1, slice_exp.shape[0], squeeze=False, sharey=True)
    fig_f.set_size_inches(10, 5)
    fig_f.suptitle(f"FFT of standardized distance slices, at frequency {f:.0f}Hz")
    for m in range(slice_exp.shape[0]):
        slice_exp_norm = deepcopy(slice_exp[m])
        slice_the_norm = deepcopy(slice_the[m])
        slice_exp_norm -= np.mean(slice_exp_norm)
        slice_the_norm -= np.mean(slice_the_norm)

        n = max(len(slice_exp_norm), 1000)
        freqs = np.fft.rfftfreq(n, d=dist[1] - dist[0])  # unit: 1/cm
        fft_exp = np.abs(np.fft.rfft(slice_exp_norm, n=n))
        fft_exp /= np.sum(fft_exp)
        fft_theory = np.abs(np.fft.rfft(slice_the_norm, n=n))
        fft_theory /= np.sum(fft_theory)

        axs_f[0, m].plot(freqs, fft_theory, label="theoretical")
        axs_f[0, m].plot(freqs, fft_exp, label="measured")
        axs_f[0, m].set_title(f"mic{m}")
        axs_f[0, m].legend(loc="upper right")
    return fig_f

valid_freqs = df_freq
plot_freqs = [valid_freqs[0], valid_freqs[len(valid_freqs) // 2], valid_freqs[-2]]
for i, f in enumerate(df_freq):
    slice_exp = df_matrix[:, i, :]
    if np.any(np.isnan(slice_exp)):
        slice_exp = fill_nans(slice_exp, df_dist)
    slice_the = get_dist_slice_theory(f, df_dist).T
    
    fig_f = plot_ffts(
        slice_exp=slice_exp,
        slice_the=slice_the,
        dist=df_dist,
    )
    break

In [ ]:
from calibration import fit_distance_slice
import time

fitting_results = pd.DataFrame(
    columns=["frequency", "mic", "absorption", "gains", "offset", "method"]
)

chosen_mics = range(4)
methods = ["brute", "minimize"]
times = {m: [] for m in methods}

for i_f, f in enumerate(df_freq):
    fig, axs = plt.subplots(1, len(chosen_mics), sharey=True, squeeze=False)
    fig.set_size_inches(5 * len(chosen_mics), 5)
    fig.suptitle(f"{f}Hz")

    slice_exp = df_matrix[:, i_f, :]
    if np.any(np.isnan(slice_exp)):
        slice_exp = fill_nans(slice_exp, df_dist)

    # do one fit
    coeffs_glob, slice_calib_glob, cost = fit_distance_slice(
        slice_exp.T[:, chosen_mics],
        df_dist,
        yaw_deg=0,
        frequency=f,
        chosen_mics=chosen_mics,
        method="minimize",
        optimize_absorption=True,
    )
    print(f"absorption at {f}: {coeffs_glob[0]:.2f}")
    # do separate fits
    for i, mic_idx in enumerate(chosen_mics):
        axs[0, i].plot(
            df_dist, slice_exp[mic_idx, :], label="experimental", color="C0"
        )
        axs[0, i].plot(
            df_dist,
            slice_calib_glob[:, i],
            ls=":",
            color="C0",
            label="calibrated one-shot",
        )
        axs[0, i].set_title(f"mic{mic_idx}")

        fitting_results.loc[len(fitting_results), :] = dict(
            frequency=f,
            mic=mic_idx,
            absorption=coeffs_glob[0],
            gains=coeffs_glob[2 + i],
            offset=coeffs_glob[1],
            method="one-shot",
        )

        for i_m, method in enumerate(methods):
            t0 = time.time()
            coeffs, slice_calib, cost = fit_distance_slice(
                slice_exp.T[:, [mic_idx]],
                df_dist,
                yaw_deg=0,
                frequency=f,
                chosen_mics=[mic_idx],
                method=method,
                optimize_absorption=True,
            )
            times[method].append(time.time() - t0)
            fitting_results.loc[len(fitting_results), :] = dict(
                frequency=f,
                mic=mic_idx,
                absorption=coeffs[0],
                gains=coeffs[2:],
                offset=coeffs[1],
                method=method,
            )
            axs[0, i].plot(
                df_dist,
                slice_calib[:, 0],
                label=f"calibrated {method}",
                color=f"C{i_m+1}",
            )
    axs[0, i].legend()
    break
print(f"average time {method}: {np.mean(times[method]):.2f}s")

In [ ]:
import seaborn as sns
color_palette = "tab10"
extra_kwargs = {
    "linewidth": 0,
    "palette": color_palette,
    "style": "method",
    "hue": "mic",
    "data": fitting_results,
    "x": "frequency",
}
palette = sns.palettes.color_palette(color_palette)
fitting_results = fitting_results.apply(pd.to_numeric, errors="ignore", axis=0)
plt.figure()
sns.scatterplot(
    y="gains", **extra_kwargs,
)
for i, mic_idx in enumerate(chosen_mics):
    median_per_freq = np.nanmedian(df_matrix[i, :], axis=1)
    plt.plot(df_freq, median_per_freq, color=f'C{i}')
plt.plot([], [], color='black', label='median')
plt.legend(loc="upper left", bbox_to_anchor=[1, 1])

plt.figure()
sns.scatterplot(y="absorption", **extra_kwargs)

plt.figure()
sns.scatterplot(y="offset", **extra_kwargs)
plt.ylabel("offset [cm]")

ls = {"brute": "-", "minimize": ":", "one-shot": "--"}
for method, df in fitting_results.groupby("method"):
    medians = df.groupby(["mic"]).offset.median()
    [plt.axhline(m, color=palette[i], ls=ls[method]) for i, m in enumerate(medians)]
    plt.plot([], [], color="black", ls=ls[method], label=method)
plt.legend(loc="upper left", bbox_to_anchor=[1, 1])
pass

In [ ]:
from simulation import factor_distance_to_delta

for mic in range(4):
    print('mic', mic)
    for distance in np.arange(10, 50, step=10):
        f = factor_distance_to_delta(distance, mic)
        print(distance, f)

In [ ]:
# distance slice algorithm
from inference import get_approach_angle_fft

gt_gamma = 90 # in degrees
err_dict = {f"mic{m}": [np.nan]*len(df_freq) for m in range(df_matrix.shape[0])}
for i, f in enumerate(df_freq):
    
    d_slices = fill_nans(df_matrix[:, i, :], df_dist)
    
    plt.figure()
    for mic_idx in range(df_matrix.shape[0]):
        d_slice = d_slices[mic_idx]
        gammas, prob, *_ = get_approach_angle_fft(d_slice, f, df_dist, mic_idx,
                                              n_max=1000, bayes=False)
        gamma = gammas[np.argmax(prob)]
        plt.plot(gammas, prob)
        plt.axvline(gamma, label=f'mic{mic_idx}: $\\gamma$={gamma:.1f}', color=f'C{mic_idx}')
        err_dict[f"mic{mic_idx}"][i] = gamma - gt_gamma
        
    plt.title(f'frequency {f:.0f}Hz')
    plt.xlabel('angle of approach $\\gamma$ [deg]')
    plt.ylabel('probability')
    plt.legend()
        
fname = f"{exp_name}_{motors}"
fig, axs = plot_performance(err_dict, xs=df_freq, 
           xlabel="frequency [Hz]", ylabel="error [deg]")
save_fig(fig, f'plots/{fname}_distance_slice_performance.pdf')

# 2. study full matrices

In [ ]:
from calibration import get_calibration_function_matrix
from calibration import get_calibration_function, get_calibration_function_dict

fig, ax = plt.subplots()
calib_function_old = get_calibration_function(ax=ax)
ax.set_title("old calibration")

fig, ax = plt.subplots()
filter_dict = {
    'exp_name': exp_name, 
    'motors': 0,
    'mic_type': mic_type
}
calib_function = get_calibration_function_dict(**filter_dict, ax=ax)
ax.set_ylim(1, 10)
ax.set_yscale("linear")
ax.set_title("new calibration")

fig, ax = plt.subplots()
calib_function_median = wall_detector.get_calib_function(method="median", ax=ax)
ax.set_yscale("linear")
ax.set_ylim(0, 5)
ax.set_title("new calibration, median")

fig, ax = plt.subplots()
calib_function_fit = wall_detector.get_calib_function(method="fit", ax=ax)
ax.set_yscale("linear")
ax.set_ylim(0, 3)
ax.set_title("new calibration, fit")
#
fig, ax = plt.subplots()
calib_function_fit_one = wall_detector.get_calib_function(method="fit-one", ax=ax)
ax.set_yscale("linear")
ax.set_ylim(0, 3)
ax.set_title("new calibration, fit one")

In [ ]:
from simulation import get_df_theory
from wall_detector import normalize_df_matrix

min_value = np.inf
max_value = -np.inf

distances_grid = np.arange(0, max(df_dist) + 1)
distances_idx = np.argmin(np.abs(distances_grid[:, None] - df_dist[None, :]), axis=0)

results = pd.DataFrame(columns=["normalization", "matrix", "values"])
method_dict = {
    'raw': "",
    'theoretical': "",
    'calibration-median': calib_function_median,
    'calibration-fit': calib_function_fit,
    'calibration-fit-one': calib_function_fit_one,
    #'calibration-online': "calibration-online",
    #'standardize': "standardize",
    #'zero_mean': "zero_mean",
    #'normalize': "normalize",
    #'calibration-offline-old': calib_function_old,
}

df_theory_pruned = get_df_theory(df_freq, df_dist, chosen_mics)

for j, (key, method) in enumerate(method_dict.items()):
    values = None
    if key == "raw":
        df_norm = deepcopy(df_matrix)
    elif key == "theoretical":
        df_norm = deepcopy(df_theory_pruned[:, :, distances_idx])
    else:
        df_norm, values = normalize_df_matrix(
            df_matrix=df_matrix, freqs=df_freq, method=method
        )
    results.loc[len(results), :] = {
        "normalization": key,
        "matrix": df_norm,
        "values": values,
    }
    # print(key, data_type, df_norm.shape)
    min_value = min(min_value, -abs(np.min(df_norm)))
    max_value = max(max_value, abs(np.max(df_norm)))

In [ ]:
from plotting_tools import plot_df_matrix, save_fig

min_freq = min(df_freq)
max_freq = max(df_freq)

min_value = None
max_value = None

fname = f"{exp_name}_{motors}"

for normalization, df in results.groupby("normalization", sort=False):

    matrix = df.iloc[0].matrix
    n_mics = matrix.shape[0]
    fig, axs = plt.subplots(1, n_mics, sharey=True)
    fig.set_size_inches(n_mics*5, 5)
    axs[0].set_ylabel('frequency [Hz]')
    for i in range(n_mics):
        df_the = df_theory_pruned[i, :, distances_idx].T
        df_exp = df.iloc[0].matrix[i]
        ax, im = plot_df_matrix(
            df_dist,
            df_freq,
            df_exp,
            ax=axs[i],
            min_freq=min_freq,
            max_freq=max_freq,
            vmin=min_value,
            vmax=max_value,
        )
        ax.set_title(f"mic{i} {normalization}")
        ax.set_xlabel('distance [cm]')
    add_colorbar(fig, ax, im)
    save_fig(fig, f"plots/{fname}_matrices_{normalization}.png")

# 3. Evaluate algorithm performance

In [ ]:
from inference import get_probability_cost, get_probability_bayes
from simulation import get_freq_slice_theory
import progressbar

err_df = pd.DataFrame(columns=['method', 'mic', 'distance', 'error', 'algorithm'])

distances_grid = np.arange(50)
distances = df_dist
mic_indices = range(4)
n_mics = len(mic_indices)

with progressbar.ProgressBar(max_value=len(distances)) as p:
    for i_d, distance in enumerate(distances):
        p.update(i_d)
        for i_mic, mic_idx in enumerate(mic_indices):
            for method, normalize_method in method_dict.items():
                df = results.loc[results.normalization == method]

                if method == "theoretical": 
                    freqs = wall_detector.df[wall_detector.df.distance==distance].frequency.unique()
                    slice_exp = get_freq_slice_theory(freqs, distance)
                    slice_exp = slice_exp.T
                    stds = [1]*slice_exp.shape[0]
                else:
                    slice_exp, freqs, stds = wall_detector.get_frequency_slice(
                        distance, normalize_method=normalize_method)

                slice_exp = fill_nans(slice_exp, freqs)
                slice_exp = slice_exp[mic_idx]

                proba_cost = get_probability_cost(
                    slice_exp, freqs, 
                    distances_grid, mic_idx=mic_idx
                )
                distances_bayes, proba_bayes = get_probability_bayes(
                    slice_exp,
                    freqs, 
                    mic_idx=mic_idx,
                    distance_range=[min(distances_grid), max(distances_grid)],
                    sigma=stds[mic_idx]
                )

                for algo, proba, distances_here in zip(
                    ["cost", "bayes"],
                    [proba_cost, proba_bayes],
                    [distances_grid, distances_bayes],
                ):
                    d_idx = np.argmax(proba)
                    d = distances_here[d_idx]
                    err_df.loc[len(err_df), :] = {
                        'error': d - distance,
                        'mic': mic_idx,
                        'distance': distance,
                        'method': method, 
                        'algorithm': algo
                    }

In [ ]:
from plotting_tools import plot_performance
titles = {
    'cost': 'optimization-based method',
    'bayes': 'FFT-based method'
}
err_df = err_df.apply(pd.to_numeric, axis=0, errors='ignore')

fname = f"{exp_name}_{motors}"
for key, df in err_df.groupby("algorithm"):
    for mic, df_mic in df.groupby("mic"):
        absolute_err = pd.pivot_table(df_mic, index="distance", values="error", columns="method")
        fig, axs = plot_performance(absolute_err, xs=distances, xlabel="distance [cm]", ylabel="absolute error [cm]")
        axs[0, 1].set_xlim(-1, max(distances))
        axs[0, 0].set_title(f'mic{mic}, {titles[key]}')
        axs[0, 0].grid()
        
        fname_here = f"plots/{fname}_{key}_mic{mic}_frequency_performance.png"
        save_fig(fig, fname_here)

In [ ]:
from inference import get_probability_cost, get_probability_bayes
from simulation import get_freq_slice_theory
from matplotlib.ticker import FormatStrFormatter

mic_idx = 1

chosen_methods = ["raw", "calibration-median", "theoretical"]
plot_combis = [{
    'distance': 11,
    'algorithms': ["cost", "bayes"]
}]
    
for plot_combi in plot_combis: 
    distance = plot_combi.get('distance')
    algorithms = plot_combi.get('algorithms')
    fname = f"{exp_name}_{motors}_{distance:.0f}cm"
    
    fig_slice, axs_slice = plt.subplots(len(chosen_methods), n_mics, sharex=True, sharey=True)
    fig_slice.set_size_inches(3*n_mics, 2*len(chosen_methods))
    
    fig_algos = {}
    axs_algos = {}
    for algo in algorithms:
        fig_algo, axs_algo = plt.subplots(len(chosen_methods), n_mics, sharex=True, sharey='row')
        fig_algo.set_size_inches(3*n_mics, 2*len(chosen_methods))
        fig_algos[algo] = fig_algo
        axs_algos[algo] = axs_algo
    
    for i_mic, mic_idx in enumerate(mic_indices):
        for i_method, method in enumerate(chosen_methods):
            df = results.loc[results.normalization == method]
            if method == "theoretical": 
                freqs = np.sort(wall_detector.df[wall_detector.df.distance==distance].frequency.unique())
                slice_exp = get_freq_slice_theory(freqs, distance)
                slice_exp = slice_exp.T
                stds = [1]*slice_exp.shape[0]
            else:
                normalize_method = method_dict[method]
                slice_exp, freqs, stds = wall_detector.get_frequency_slice(
                    distance, normalize_method=normalize_method)

            slice_exp = fill_nans(slice_exp, freqs)
            slice_exp = slice_exp[mic_idx]

            # doing this here for plotting reasons. Doesn't change performance as
            # this is done again in get_probability_cost
            slice_exp -= np.nanmean(slice_exp)
            slice_exp /= np.nanstd(slice_exp)

            axs_slice[i_method, i_mic].plot(freqs, slice_exp, label=method, color=f"C{i_method}")
            axs_slice[i_method, i_mic].set_xlim(min_freq, max_freq)
            axs_slice[i_method, 0].set_ylabel(f'{method}')
            axs_slice[0, i_mic].set_title(f"mic{mic_idx}")
            
            for algo in algorithms:
                axs_algo = axs_algos[algo]
                
                if algo == "cost":
                    proba = get_probability_cost(
                        slice_exp, freqs, 
                        distances_grid, mic_idx=mic_idx
                    )
                    distances_here = distances_grid
                elif algo == "bayes":
                    distances_here, proba = get_probability_bayes(
                        slice_exp,
                        freqs, 
                        mic_idx=mic_idx,
                        distance_range=[min(distances_grid), max(distances_grid)],
                        sigma=stds[mic_idx]
                    )

                d_idx = np.argmax(proba)
                d = distances_here[d_idx]

                axs_algo[i_method, i_mic].semilogy(distances_here, proba, color=f"C{i_method}")
                axs_algo[i_method, i_mic].axvline(x=d, color=f"C{i_method}", label=f'estimate')
                axs_algo[i_method, i_mic].axvline(x=distance, color="black", ls=":", label=f'real: {distance:.0f}cm')
                axs_algo[i_method, i_mic].set_xlim(min(distances_here)-1, max(distances_here)+1)

                axs_algo[0, i_mic].set_title(f"mic{mic_idx}")
                axs_algo[i_method, 0].set_ylabel(f'{method}')

    fname_here = f'plots/{fname}_slice.png'
    save_fig(fig_slice, fname_here)

    for algo in algorithms:
        axs_algos[algo][-1, -1].legend(loc='upper right')
        fname_here = f'plots/{fname}_{algo}.png'
        save_fig(fig_algos[algo], fname_here)